# CSC 466 Project Codebook: Question 3 Classification

Brendan Callender, Andrew Kerr, Martin Hsu, Sophia Chung

In [87]:
import pandas as pd
import numpy as np
import os
import wget
import zipfile

from induceC45 import *
from classify import *
from validation import *
from knn import *
from randomforest import *

## Data Load

In [88]:
if not os.path.exists("data/beneficiary/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv"):
    wget.download("https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/synpufs/downloads/de1_0_2008_beneficiary_summary_file_sample_1.zip", "de1_0_2008_beneficiary_summary_file_sample_1.zip")
    with zipfile.ZipFile("de1_0_2008_beneficiary_summary_file_sample_1.zip", 'r') as zip_ref:
        zip_ref.extractall("data/beneficiary")
    os.remove("de1_0_2008_beneficiary_summary_file_sample_1.zip")
if not os.path.exists("data/beneficiary/DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv"):
    wget.download("https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/synpufs/downloads/de1_0_2009_beneficiary_summary_file_sample_1.zip", "de1_0_2009_beneficiary_summary_file_sample_1.zip")
    with zipfile.ZipFile("de1_0_2009_beneficiary_summary_file_sample_1.zip", 'r') as zip_ref:
        zip_ref.extractall("data/beneficiary")
    os.remove("de1_0_2009_beneficiary_summary_file_sample_1.zip")
if not os.path.exists("data/beneficiary/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv"):
    wget.download("https://www.cms.gov/research-statistics-data-and-systems/statistics-trends-and-reports/synpufs/downloads/de1_0_2010_beneficiary_summary_file_sample_20.zip", "de1_0_2010_beneficiary_summary_file_sample_1.zip")
    with zipfile.ZipFile("de1_0_2010_beneficiary_summary_file_sample_1.zip", 'r') as zip_ref:
        zip_ref.extractall("data/beneficiary")
    os.remove("de1_0_2010_beneficiary_summary_file_sample_1.zip")

In [89]:
if not os.path.exists("data/inpatient/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv"):
    wget.download("https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/synpufs/downloads/de1_0_2008_to_2010_inpatient_claims_sample_1.zip", "de1_0_2008_to_2010_inpatient_claims_sample_1.zip")
    with zipfile.ZipFile("de1_0_2008_to_2010_inpatient_claims_sample_1.zip", 'r') as zip_ref:
        zip_ref.extractall("data/inpatient")
    os.remove("de1_0_2008_to_2010_inpatient_claims_sample_1.zip")

## Data Wrangling

In [2]:
bene_2008 = pd.read_csv("data/beneficiary/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv")
bene_2008["Year"] = pd.to_datetime("12-31-2008")
bene_2008["BENE_BIRTH_DT"] = pd.to_datetime(bene_2008["BENE_BIRTH_DT"].astype(str))
bene_2008 = bene_2008.set_index("DESYNPUF_ID")
bene_2008

,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,Year
DESYNPUF_ID,,,,,,,,,,,,,,,,,,,,,
00013D2EFD8E45D1,1923-05-01,NaN,1,1,0,26,950,12,12,12,...,0.0,0.0,0.0,50.0,10.0,0.0,0.0,0.0,0.0,2008-12-31
00016F745862898F,1943-01-01,NaN,1,1,0,39,230,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,700.0,240.0,0.0,2008-12-31
0001FDD721E223DC,1936-09-01,NaN,2,1,0,39,280,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008-12-31
00021CA6FF03E670,1941-06-01,NaN,1,5,0,6,290,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008-12-31
00024B3D2352D2D0,1936-08-01,NaN,1,1,0,52,590,12,12,0,...,0.0,0.0,0.0,30.0,40.0,0.0,220.0,80.0,0.0,2008-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFFD7C464C94D136,1943-07-01,NaN,2,1,0,54,0,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008-12-31
FFFE3ED9582AD46E,1916-06-01,NaN,2,1,0,6,150,12,12,0,...,0.0,0.0,0.0,1570.0,780.0,0.0,1370.0,350.0,0.0,2008-12-31
FFFE7AB5D7F1725D,1939-04-01,NaN,2,1,0,23,490,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008-12-31


In [3]:
bene_2009 = pd.read_csv("data/beneficiary/DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv")
bene_2009["Year"] = pd.to_datetime("12-31-2009")
bene_2009["BENE_BIRTH_DT"] = pd.to_datetime(bene_2009["BENE_BIRTH_DT"].astype(str))
bene_2009 = bene_2009.set_index("DESYNPUF_ID")
bene_2009

,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,Year
DESYNPUF_ID,,,,,,,,,,,,,,,,,,,,,
00013D2EFD8E45D1,1923-05-01,NaN,1,1,0,26,950,12,12,12,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,2009-12-31
00016F745862898F,1943-01-01,NaN,1,1,0,39,230,12,12,0,...,36000.0,3204.0,0.0,60.0,70.0,0.0,1350.0,530.0,100.0,2009-12-31
0001FDD721E223DC,1936-09-01,NaN,2,1,0,39,280,12,12,0,...,0.0,0.0,0.0,30.0,50.0,0.0,20.0,0.0,0.0,2009-12-31
00021CA6FF03E670,1941-06-01,NaN,1,5,0,6,290,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,90.0,10.0,0.0,2009-12-31
00024B3D2352D2D0,1936-08-01,NaN,1,1,0,52,590,12,12,2,...,0.0,0.0,0.0,90.0,40.0,0.0,410.0,140.0,0.0,2009-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFFD7C464C94D136,1943-07-01,NaN,2,1,0,54,0,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,40.0,0.0,2009-12-31
FFFE3ED9582AD46E,1916-06-01,NaN,2,1,0,6,150,12,12,0,...,2000.0,1068.0,0.0,3240.0,1390.0,0.0,2510.0,620.0,0.0,2009-12-31
FFFE7AB5D7F1725D,1939-04-01,NaN,2,1,0,23,490,12,12,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009-12-31


In [4]:
bene_2010 = pd.read_csv("data/beneficiary/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv")
bene_2010["Year"] = pd.to_datetime("12-31-2010")
bene_2010["BENE_BIRTH_DT"] = pd.to_datetime(bene_2010["BENE_BIRTH_DT"].astype(str))
bene_2010 = bene_2010.set_index("DESYNPUF_ID")
bene_2010

,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,Year
DESYNPUF_ID,,,,,,,,,,,,,,,,,,,,,
00013D2EFD8E45D1,1923-05-01,NaN,1,1,0,26,950,12,12,12,...,4000.0,1100.0,0.0,0.0,0.0,0.0,90.0,30.0,0.0,2010-12-31
00016F745862898F,1943-01-01,NaN,1,1,Y,39,230,12,12,0,...,16000.0,1100.0,0.0,0.0,0.0,0.0,930.0,150.0,0.0,2010-12-31
0001FDD721E223DC,1936-09-01,NaN,2,1,0,39,280,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-12-31
00021CA6FF03E670,1941-06-01,NaN,1,5,0,6,290,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-12-31
00024B3D2352D2D0,1936-08-01,NaN,1,1,0,52,590,9,12,12,...,0.0,0.0,0.0,40.0,0.0,0.0,590.0,60.0,0.0,2010-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFFD7C464C94D136,1943-07-01,NaN,2,1,0,54,0,12,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,10.0,0.0,2010-12-31
FFFE3ED9582AD46E,1916-06-01,20101101.0,2,1,0,6,150,12,12,0,...,0.0,0.0,0.0,360.0,280.0,0.0,1040.0,330.0,0.0,2010-12-31
FFFE7AB5D7F1725D,1939-04-01,NaN,2,1,0,23,490,12,12,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-12-31


In [5]:
ip_claims = pd.read_csv("data/inpatient/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv")
ip_claims["Claim Count"] = 1
ip_claims = ip_claims[["DESYNPUF_ID", "Claim Count", "CLM_PMT_AMT"]].groupby("DESYNPUF_ID").sum()
ip_claims

,Claim Count,CLM_PMT_AMT
DESYNPUF_ID,,
00013D2EFD8E45D1,1,4000.0
00016F745862898F,4,52000.0
00052705243EA128,1,14000.0
0007F12A492FD25D,4,53000.0
000B97BA2314E971,1,2000.0
...,...,...
FFF5F9218AB04A41,2,9000.0
FFF8B49458F9FFA7,2,13000.0
FFFA950301FCA748,1,7000.0


In [6]:
benes = pd.concat([bene_2010, bene_2009, bene_2008])
benes = benes[~benes.index.duplicated(keep='first')]
benes = benes.merge(how = "right", right=ip_claims, left_index=True, right_index=True)
benes["Est Age in Days"] = (benes["Year"] - benes["BENE_BIRTH_DT"]).apply(lambda x: x.days)
benes["BENE_RACE_CD"] = benes["BENE_RACE_CD"].astype(str)
benes = benes[["Est Age in Days", "CLM_PMT_AMT", "Claim Count", "BENE_RACE_CD", "BENE_SEX_IDENT_CD",
               "SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN",
               "SP_CNCR", "SP_COPD", "SP_DEPRESSN",
               "SP_DIABETES", "SP_ISCHMCHT", "SP_OSTEOPRS",
               "SP_RA_OA", "SP_STRKETIA"]]
benes.loc[:, ["SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN", "SP_CNCR", "SP_COPD", "SP_DEPRESSN", "SP_DIABETES", "SP_ISCHMCHT", "SP_OSTEOPRS", "SP_RA_OA", "SP_STRKETIA"]] = benes.loc[:, ["SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN", "SP_CNCR", "SP_COPD", "SP_DEPRESSN", "SP_DIABETES", "SP_ISCHMCHT", "SP_OSTEOPRS", "SP_RA_OA", "SP_STRKETIA"]].mod(2)
benes["BENE_SEX_IDENT_CD"] = benes["BENE_SEX_IDENT_CD"].mod(2).astype(str)
benes

,Est Age in Days,CLM_PMT_AMT,Claim Count,BENE_RACE_CD,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA
DESYNPUF_ID,,,,,,,,,,,,,,,,
00013D2EFD8E45D1,32021,4000.0,1,1,1,0,1,0,0,0,0,0,0,1,0,0
00016F745862898F,24836,52000.0,4,1,1,1,1,1,0,0,0,1,0,0,0,0
00052705243EA128,28003,14000.0,1,1,1,0,0,1,1,0,0,1,1,0,0,0
0007F12A492FD25D,33359,53000.0,4,2,0,0,1,1,0,1,1,1,1,0,0,0
000B97BA2314E971,26572,2000.0,1,1,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFF5F9218AB04A41,20910,9000.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0
FFF8B49458F9FFA7,33237,13000.0,2,1,0,1,1,1,0,0,0,0,1,0,0,0
FFFA950301FCA748,18811,7000.0,1,1,1,0,0,0,0,0,1,1,1,1,0,0


In [7]:
bene_cls = benes.copy()
bene_cls['CC Group'] = (1 * ((benes['SP_DEPRESSN'] == 1) & (benes['SP_DIABETES'] == 0) & (benes['SP_ISCHMCHT'] == 0)) +
                        2 * ((benes['SP_DEPRESSN'] == 0) & (benes['SP_DIABETES'] == 1) & (benes['SP_ISCHMCHT'] == 0)) +
                        3 * ((benes['SP_DEPRESSN'] == 0) & (benes['SP_DIABETES'] == 0) & (benes['SP_ISCHMCHT'] == 1)) +
                        4 * ((benes['SP_DEPRESSN'] == 0) & (benes['SP_DIABETES'] == 1) & (benes['SP_ISCHMCHT'] == 1)) +
                        5 * ((benes['SP_DEPRESSN'] == 1) & (benes['SP_DIABETES'] == 0) & (benes['SP_ISCHMCHT'] == 1)) +
                        6 * ((benes['SP_DEPRESSN'] == 1) & (benes['SP_DIABETES'] == 1) & (benes['SP_ISCHMCHT'] == 0)) +
                        7 * ((benes['SP_DEPRESSN'] == 1) & (benes['SP_DIABETES'] == 1) & (benes['SP_ISCHMCHT'] == 1))).astype(str)
bene_cls.loc[:, ["SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN", "SP_CNCR", "SP_COPD", "SP_DEPRESSN", "SP_DIABETES", "SP_ISCHMCHT", "SP_OSTEOPRS", "SP_RA_OA", "SP_STRKETIA"]] = bene_cls.loc[:, ["SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN", "SP_CNCR", "SP_COPD", "SP_DEPRESSN", "SP_DIABETES", "SP_ISCHMCHT", "SP_OSTEOPRS", "SP_RA_OA", "SP_STRKETIA"]].astype(str)
bene_cls = bene_cls.drop(['SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT'], axis=1)
bene_cls

,Est Age in Days,CLM_PMT_AMT,Claim Count,BENE_RACE_CD,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,CC Group
DESYNPUF_ID,,,,,,,,,,,,,,
00013D2EFD8E45D1,32021,4000.0,1,1,1,0,1,0,0,0,1,0,0,0
00016F745862898F,24836,52000.0,4,1,1,1,1,1,0,0,0,0,0,2
00052705243EA128,28003,14000.0,1,1,1,0,0,1,1,0,0,0,0,4
0007F12A492FD25D,33359,53000.0,4,2,0,0,1,1,0,1,0,0,0,7
000B97BA2314E971,26572,2000.0,1,1,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFF5F9218AB04A41,20910,9000.0,2,1,0,0,0,0,0,0,0,0,0,0
FFF8B49458F9FFA7,33237,13000.0,2,1,0,1,1,1,0,0,0,0,0,3
FFFA950301FCA748,18811,7000.0,1,1,1,0,0,0,0,0,1,0,0,7


In [55]:
bene_samp = bene_cls.sample(frac=0.1)

## K-Nearest Neighbors Classification

In [19]:
bene_knn = bene_cls.astype(int).reset_index(drop=True)
bene_knn["CC Group"] = bene_knn["CC Group"].astype(str)

In [20]:
bene_knn

,Est Age in Days,CLM_PMT_AMT,Claim Count,BENE_RACE_CD,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,CC Group
0,32021,4000,1,1,1,0,1,0,0,0,1,0,0,0
1,24836,52000,4,1,1,1,1,1,0,0,0,0,0,2
2,28003,14000,1,1,1,0,0,1,1,0,0,0,0,4
3,33359,53000,4,2,0,0,1,1,0,1,0,0,0,7
4,26572,2000,1,1,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37775,20910,9000,2,1,0,0,0,0,0,0,0,0,0,0
37776,33237,13000,2,1,0,1,1,1,0,0,0,0,0,3
37777,18811,7000,1,1,1,0,0,0,0,0,1,0,0,7
37778,27058,10000,1,1,0,0,0,0,0,0,0,0,0,0


In [21]:
pred = knn_predict(bene_knn, 10, classvar="CC Group")

In [22]:
pred

,Est Age in Days,CLM_PMT_AMT,Claim Count,BENE_RACE_CD,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,CC Group,pred
0,32021,4000,1,1,1,0,1,0,0,0,1,0,0,0,3
1,24836,52000,4,1,1,1,1,1,0,0,0,0,0,2,4
2,28003,14000,1,1,1,0,0,1,1,0,0,0,0,4,4
3,33359,53000,4,2,0,0,1,1,0,1,0,0,0,7,4
4,26572,2000,1,1,1,0,0,0,0,0,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37775,20910,9000,2,1,0,0,0,0,0,0,0,0,0,0,0
37776,33237,13000,2,1,0,1,1,1,0,0,0,0,0,3,7
37777,18811,7000,1,1,1,0,0,0,0,0,1,0,0,7,1
37778,27058,10000,1,1,0,0,0,0,0,0,0,0,0,0,0


In [43]:
results = report(pred, 'CC Group', 'pred')
results_table = pd.DataFrame()
results_table['Overall Precision:'] = results['precision']
results_table['Overall Recall:'] = results['recall']
results_table.index.name = None
results_table = results_table.T

report_text = ("KNN Classifier, k=10:"
               f"\n\nConfusion Matrix:"
               f"\n" + results['conf_matrix'].to_string() +
               f"\n\nRecords Classified: {results['n']}"
               f"\nRecords Correctly Classified: "
               f"{results['n_correct']}"
               f"\nRecords Incorrectly Classified: "
               f"{results['n_incorrect']}"
               f"\nOverall Accuracy: {results['accuracy']:0.3f}"
               f"\nOverall Error Rate: "
               f"{results['error_rate']:0.3f}\n"
               + results_table.to_string())

In [44]:
print(report_text)

KNN Classifier, k=10:

Confusion Matrix:
obs      0    1     2     3     4    5    6     7
pred                                             
0     4441  712  1062  2204  1534  624  369   704
1      107   21    37    88    64   29   12    39
2      303   56   185   319   378  101   83   221
3     1342  335   641  1217  1315  451  257   687
4     1344  393  1080  1803  3291  823  547  2219
5      137   31    78   160   174   68   37   129
6       36   17    30    46    71   24   16    51
7      389  175   392   673  1573  385  236  1414

Records Classified: 37780
Records Correctly Classified: 10653
Records Incorrectly Classified: 27127
Overall Accuracy: 0.282
Overall Error Rate: 0.718
                           0         1         2         3         4         5         6         7
Overall Precision:  0.381202  0.052897  0.112394  0.194876  0.286174  0.083538  0.054983  0.270002
Overall Recall:     0.548339  0.012069  0.052782  0.186943  0.391786  0.027146  0.010276  0.258785


## Random Forest Classification

In [80]:
D = bene_cls.drop("CC Group", axis=1)
C = bene_cls["CC Group"]
A = {a: list(D[a].unique()) for a in list(D.columns)}

In [81]:
trees = random_forest(D, A, C, 5, 1000, 5, 0.02)

In [82]:
rf_pred = rf_predict(D, trees)
rf_pred["CC Group"] = C

In [83]:
rf_pred

,Est Age in Days,CLM_PMT_AMT,Claim Count,BENE_RACE_CD,BENE_SEX_IDENT_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,pred,CC Group
DESYNPUF_ID,,,,,,,,,,,,,,,
00013D2EFD8E45D1,32021,4000.0,1,1,1,0,1,0,0,0,1,0,0,0,0
00016F745862898F,24836,52000.0,4,1,1,1,1,1,0,0,0,0,0,0,2
00052705243EA128,28003,14000.0,1,1,1,0,0,1,1,0,0,0,0,0,4
0007F12A492FD25D,33359,53000.0,4,2,0,0,1,1,0,1,0,0,0,4,7
000B97BA2314E971,26572,2000.0,1,1,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFF5F9218AB04A41,20910,9000.0,2,1,0,0,0,0,0,0,0,0,0,0,0
FFF8B49458F9FFA7,33237,13000.0,2,1,0,1,1,1,0,0,0,0,0,0,3
FFFA950301FCA748,18811,7000.0,1,1,1,0,0,0,0,0,1,0,0,0,7


In [84]:
results = report(rf_pred, 'CC Group', 'pred')
results_table = pd.DataFrame()
results_table['Overall Precision:'] = results['precision']
results_table['Overall Recall:'] = results['recall']
results_table.index.name = None
results_table = results_table.T

report_text = ("Random Forest Classifier:\n"
               "5 features, 1000 sample size per tree\n"
               "5 trees, 0.02 threshold"
               f"\n\nConfusion Matrix:"
               f"\n" + results['conf_matrix'].to_string() +
               f"\n\nRecords Classified: {results['n']}"
               f"\nRecords Correctly Classified: "
               f"{results['n_correct']}"
               f"\nRecords Incorrectly Classified: "
               f"{results['n_incorrect']}"
               f"\nOverall Accuracy: {results['accuracy']:0.3f}"
               f"\nOverall Error Rate: "
               f"{results['error_rate']:0.3f}\n"
               + results_table.to_string())

In [85]:
print(report_text)

Random Forest Classifier:
5 features, 1000 samples per tree
5 trees, 0.02 threshold

Confusion Matrix:
obs      0     1     2     3     4     5     6     7
pred                                                
0     7456  1500  2749  5313  5555  1890  1147  3104
1        2     5    10     9    25     3     5    19
2       60    30    69    98   124    39    21    98
3      142    44   126   249   412   111    56   267
4      364   141   450   687  1879   372   258  1458
5       21     4     9    27    46    18     4    44
6        3     0     1     3     3     1     2    10
7       51    16    91   124   356    71    64   464

Records Classified: 37780
Records Correctly Classified: 10142
Records Incorrectly Classified: 27638
Overall Accuracy: 0.268
Overall Error Rate: 0.732
                           0         1         2         3         4         5         6         7
Overall Precision:  0.259664  0.064103  0.128015  0.176972  0.334997  0.104046  0.086957  0.375101
Overall Recall:   